In [1]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
! pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.1 MB/s eta 0:00:00


In [9]:
# Restart the kernel after running this cell.
!pip install -U bitsandbytes

In [2]:
! pip install jsonlines

In [1]:
!pip install accelerate
!pip install -U bitsandbytes
!pip install transformers

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import PrefixTuningConfig, get_peft_model
from datasets import Dataset
import jsonlines

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Load the model in 4-bit precision
    bnb_4bit_use_double_quant=True,  # Use double quantization for better efficiency
    bnb_4bit_quant_type="nf4",  # Use normalized float 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16  # Compute dtype for 4-bit models
)

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    quantization_config=quantization_config,
    device_map="auto"  # Automatically map the model to available hardware (CPU/GPU)
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [5]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set the padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding token

# Move the model to the GPU
model.to(device)

# Define prefix tuning configuration
prefix_tuning_config = PrefixTuningConfig(
    task_type="CAUSAL_LM",  # Task type (causal language modeling)
    num_virtual_tokens=10,  # Number of virtual tokens (prefix length)
)

# Apply prefix tuning to the model
model = get_peft_model(model, prefix_tuning_config)

# Load the dataset from the uploaded file
data = []
with open("book_dataset.jsonl", "r", encoding="utf-8") as f:
    for item in jsonlines.Reader(f):
        data.append(item)

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict({
    "instruction": [item["instruction"] for item in data],
    "input": [item["input"] for item in data],
    "output": [item["output"] for item in data]
})

# Preprocessing function
def preprocess_function(examples):
    prompts = [
        f"Instruction: {inst}\nInput: {inp}\nOutput: {out}"
        for inst, inp, out in zip(examples["instruction"], examples["input"], examples["output"])
    ]
    tokenized_inputs = tokenizer(prompts, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].clone()

    # Move tokenized inputs to the GPU
    for key in tokenized_inputs:
        tokenized_inputs[key] = tokenized_inputs[key].to(device)

    return tokenized_inputs

# Apply preprocessing to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",  # Directory to save the model
    per_device_train_batch_size=4,  # Batch size
    num_train_epochs=3,  # Number of epochs
    logging_dir="./logs",  # Directory for logs
    save_steps=500,  # Save model every 500 steps
    save_total_limit=2,  # Keep only the last 2 models
    fp16=True,  # Use mixed precision (if supported)
    gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps
    logging_steps=10,  # Log every 10 steps
    evaluation_strategy="no",  # No evaluation
    load_best_model_at_end=False,  # Disable loading the best model at the end
)
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

Using device: cuda


Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

<ipython-input-5-e17f50fe9f21>:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
10,8.380400
20,7.954600
30,7.553000
40,7.117900
50,6.768500
60,6.326100
70,5.985700
80,5.596700
90,5.241400
100,4.930800


Step,Training Loss
10,8.380400
20,7.954600
30,7.553000
40,7.117900
50,6.768500
60,6.326100
70,5.985700
80,5.596700
90,5.241400
100,4.930800


TrainOutput(global_step=1143, training_loss=2.8118392476259566, metrics={'train_runtime': 2322.2756, 'train_samples_per_second': 3.934, 'train_steps_per_second': 0.492, 'total_flos': 4.332082227314688e+16, 'train_loss': 2.8118392476259566, 'epoch': 3.0})

In [ ]:
"""from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("./results")  # Load your trained model

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move the model to the GPU
model.to(device)

# Set the padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding token

# Function to generate a response
def generate_response(prompt, max_length=100):
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate output
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,  # Maximum length of the generated text
        num_return_sequences=1,  # Number of sequences to generate
        no_repeat_ngram_size=2,  # Prevent repetition of n-grams
        top_k=50,  # Top-k sampling
        top_p=0.95,  # Top-p (nucleus) sampling
        temperature=0.7,  # Sampling temperature
        do_sample=True,  # Enable sampling
    )

    # Decode the generated text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Interactive loop to ask questions
while True:
    # Get user input
    user_input = input("You: ")

    # Exit condition
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting...")
        break

    # Format the prompt
    prompt = f"Instruction: Continue this story or summarize.\nInput: {user_input}\nOutput:"

    # Generate and print the model's response
    response = generate_response(prompt)
    print(f"Model: {response}\n")"""

In [ ]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import PeftModel, PeftConfig  # Import PeftModel and PeftConfig

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

# Load the PEFT configuration
peft_config = PeftConfig.from_pretrained("/content/results/checkpoint-1143")  # Load PEFT config

# Load the base model
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    # ... other configurations if needed (like quantization_config) ...
)

# Load the PEFT adapter weights into the base model
model = PeftModel.from_pretrained(model, "/content/results/checkpoint-1143", config=peft_config)  # Apply PEFT

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move the model to the GPU
model.to(device)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding token

# Function to generate a response
def generate_response(prompt, max_length=100):
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate output
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,  # Maximum length of the generated text
        num_return_sequences=1,  # Number of sequences to generate
        no_repeat_ngram_size=2,  # Prevent repetition of n-grams
        top_k=50,  # Top-k sampling
        top_p=0.95,  # Top-p (nucleus) sampling
        temperature=0.7,  # Sampling temperature
        do_sample=True,  # Enable sampling
    )

    # Decode the generated text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Interactive loop to ask questions
while True:
    # Get user input
    user_input = input("You: ")

    # Exit condition
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting...")
        break

    # Format the prompt
    prompt = f"Instruction: Continue this story or summarize.\nInput: {user_input}\nOutput:"

    # Generate and print the model's response
    response = generate_response(prompt)
    print(f"Model: {response}\n")